In [1]:
import os

In [2]:
%pwd

'c:\\Users\\vivek gupta\\Desktop\\Kidney_Disease_CLassification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\vivek gupta\\Desktop\\Kidney_Disease_CLassification'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/vivek34561/kidney_disease_classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="vivek34561"
os.environ["MLFLOW_TRACKING_PASSWORD"]="vivek2175@"

In [6]:
import tensorflow as tf

c:\Users\vivek gupta\Desktop\Kidney_Disease_CLassification\kidney_venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\vivek gupta\Desktop\Kidney_Disease_CLassification\kidney_venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\vivek gupta\Desktop\Kidney_Disease_CLassification\kidney_venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the r

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        # Construct path to actual dataset folder
        training_data_dir = os.path.join(
            self.config.data_ingestion.unzip_dir,
            "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone"
        )

        # You must have this field in config.yaml: 
        # evaluation:
        #   mlflow_uri: <your_uri>
        mlflow_uri = self.config.get("evaluation", {}).get("mlflow_uri", "")


        eval_config = EvaluationConfig(
            path_of_model=Path(self.config.training.trained_model_path),
            training_data=Path(training_data_dir),
            mlflow_uri=mlflow_uri,
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import tensorflow as tf
from pathlib import Path

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1. / 255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        
        # ❌ BUG: You wrote `model.evaluate()` instead of `self.model.evaluate()`
        self.score = self.model.evaluate(self.valid_generator)
        
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        mlflow.set_experiment("Default")  

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "loss": self.score[0],
                "accuracy": self.score[1]
            })

        


In [13]:
# mlflow.set_tracking_uri("https://dagshub.com/vivek34561/kidney_disease_classification.mlflow")
# print("Tracking URI:", mlflow.get_tracking_uri())


In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 148 images belonging to 2 classes.


c:\Users\vivek gupta\Desktop\Kidney_Disease_CLassification\kidney_venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.7027 - loss: 6.9108
🏃 View run dapper-bear-39 at: https://dagshub.com/vivek34561/kidney_disease_classification.mlflow/#/experiments/0/runs/3871ae2bc16141c8b5e6774baf3ea76a
🧪 View experiment at: https://dagshub.com/vivek34561/kidney_disease_classification.mlflow/#/experiments/0


In [15]:
print("Tracking URI:", mlflow.get_tracking_uri())

Tracking URI: https://dagshub.com/vivek34561/kidney_disease_classification.mlflow
